In [17]:
import numpy as np
import pandas as pd
import plotly.express as px
import nixio
from usz_neuro_conversion.common import read_nix, NixContext

In [6]:
nix_context = NixContext(
    session=5, subject=2, project="Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM"
)
nixfile = read_nix(nix_context)
# nixfile.pprint()

_general = nixfile.sections["General"]
recording_location = _general.props["Recording location"].values[0]
recording_setup = (
    _general.sections["Recording setup"].props["Recording setup EEG"].values[0]
)

_electrode_map = (
    nixfile.blocks["Data_Subject_02_Session_05"]
    .groups["Scalp EEG electrode information"]
    .data_arrays["Scalp_Electrode_EEGLAB_BESA_Coordinates"]
)

# See <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
# in x, y ,z
eeg_electrode_locations = np.ndarray(_electrode_map.shape)
_electrode_map.read_direct(eeg_electrode_locations)

In [159]:
print(recording_location)
print(recording_setup)
# print(eeg_electrode_locations)
print(_general.sections["Related publications"].props)

Schweizerische Epilepsie-Zentrum, 8008 Zurich, Switzerland
NicOne; original sampling rate, 256 Hz; passband, 0.3 to 100 Hz
[Property: {name = Publication name}, Property: {name = Publication DOI}]


In [13]:
print(len(nixfile.blocks["Data_Subject_02_Session_05"]
          .groups["Scalp EEG data"]
          .data_arrays))


46
[DataArray: {name = Scalp_EEG_Data_Trial_01, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_02, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_03, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_04, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_05, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_06, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_07, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_08, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_09, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_10, type = nix.regular_sampled.multiple_series}, DataArray: {name = Scalp_EEG_Data_Trial_11, type = nix.regular_sampled.multiple_series}, DataArray: {name 

In [160]:
# BESA map: +X is nose, +Y is left, +Z is up according to <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
eeg_electrode_locations_df = pd.DataFrame(
    eeg_electrode_locations, columns=["X", "Y", "Z"]
)
# By eye: ["F3", "F4", "C3", "C4", "P3", "P4", "O1", "O2", "F7", "F8", "T7", "T8", "P7", "P8", "FZ", "CZ", "PZ", "LPA", "RPA"]
_labels = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[0]
    .labels
)
eeg_electrode_locations_df.insert(3, "location", _labels)
print(eeg_electrode_locations_df.to_string())
fig = px.scatter_3d(
    eeg_electrode_locations_df,
    x="X",
    y="Y",
    z="Z",
    color=eeg_electrode_locations_df.index,
)
fig.show()

               X             Y          Z location
0   5.755106e+01  4.820043e+01  39.869712       F3
1   5.758403e+01 -4.814260e+01  39.891983       F4
2   3.868125e-15  6.317128e+01  56.871691       C3
3   3.867882e-15 -6.316731e+01  56.876102       C4
4  -5.755106e+01  4.820043e+01  39.869712       P3
5  -5.758403e+01 -4.814260e+01  39.891983       P4
6  -8.078401e+01  2.613301e+01  -4.001085       O1
7  -8.078401e+01 -2.613301e+01  -4.001085       O2
8   4.987138e+01  6.842334e+01  -7.489518       F7
9   4.992653e+01 -6.838359e+01  -7.485085       F8
10  5.176493e-15  8.453854e+01  -8.845083       T3
11  5.176493e-15 -8.453854e+01  -8.845083       T4
12 -4.987138e+01  6.842334e+01  -7.489518       T5
13 -4.992653e+01 -6.838359e+01  -7.485085       T6
14  6.073848e+01 -0.000000e+00  59.462904       Fz
15  5.204749e-15 -0.000000e+00  85.000000       Cz
16 -6.073848e+01 -7.438319e-15  59.462904       Pz
17  3.680313e-15  6.010408e+01 -60.104076       A1
18  3.680313e-15 -6.010408e+01 

In [161]:
recording_setup_ieeg = (
    _general.sections["Recording setup"].props["Recording setup EEG"].values[0]
)

_electrode_map_ieeg = (
    nixfile.blocks["Data_Subject_06_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays["iEEG_Electrode_MNI_Coordinates"]
)

ieeg_electrode_locations = np.ndarray(_electrode_map_ieeg.shape)
_electrode_map_ieeg.read_direct(ieeg_electrode_locations)

ieeg_electrode_locations_df = pd.DataFrame(
    ieeg_electrode_locations, columns=["X", "Y", "Z"]
)
_labels = (
    nixfile.blocks["Data_Subject_06_Session_01"]
    .groups["iEEG data"]
    .data_arrays["iEEG_Data_Trial_01"]
    .dimensions[0]
    .labels
)
ieeg_electrode_locations_df.insert(3, "location", _labels)
print(ieeg_electrode_locations_df.to_string())

KeyError: "Item not found 'Data_Subject_06_Session_01'"

In [162]:
_sampled_dimension = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[1]
)
print(f"Start at {_sampled_dimension.offset} {_sampled_dimension.unit} w.r.t. fixation")
print(f"Sampled every {_sampled_dimension.sampling_interval} {_sampled_dimension.unit}")

_events = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags scalp EEG"]
    .tags
)
_EVENTS_PER_TRIAL = 5
trials = 1
for i in range(0, trials * _EVENTS_PER_TRIAL, _EVENTS_PER_TRIAL):
    print("--")
    for j in range(_EVENTS_PER_TRIAL):
        _event = _events[i + j]
        print(
            f"Event {_event.name}: {_event.position[1]} {_event.units[0]} (w.r.t. fixation), lasted {_event.extent[1]} {_event.units[0]}"
        )

Start at -6.0 s w.r.t. fixation
Sampled every 0.005 s
--
Event Event_Fixation_ss8_c1_m2_Trial_01_Scalp_EEG: -6.0 s (w.r.t. fixation), lasted 5e-324 s
Event Event_Stimulus_ss8_xxxxxxxx_Trial_01_Scalp_EEG: -5.0 s (w.r.t. fixation), lasted 1e-323 s
Event Event_Maintenance_Trial_01_Scalp_EEG: -3.0 s (w.r.t. fixation), lasted 1.5e-323 s
Event Event_Probe_m2_H_Trial_01_Scalp_EEG: 0.0 s (w.r.t. fixation), lasted 0.0 s
Event Event_Response_c1_Trial_01_Scalp_EEG: 2.484 s (w.r.t. fixation), lasted 0.0 s


In [163]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[1]
)

SampledDimension: {index = 2}


In [164]:
ref = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[0]
    .labels
)
print(ref)
for i in range(1, 50):
    assert (
            nixfile.blocks["Data_Subject_01_Session_01"]
            .groups["Scalp EEG data"]
            .data_arrays[f"Scalp_EEG_Data_Trial_{i:02d}"]
            .dimensions[0]
            .labels
            == ref
    )
print("All trials have the same electrode order")

('F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz', 'A1', 'A2')
All trials have the same electrode order


In [165]:
from os.path import join
from datetime import datetime


def _read_session_metadata() -> pd.DataFrame:
    # This file was manually verified. Prefer the data in it over the one in the NIX file.
    trial_data_path = join("../in", "metadata", "participants.csv")
    df = pd.read_csv(
        trial_data_path,
        usecols=[
            "Participant",
            "Session",
            "Age",
            "Sex",
            "Pathology",
            "sEEG electrodes analysed",
            "Electrodes in seizure onset zone (SOZ)",
            "Handedness",
            "Date",
            "Session Start Time",
            "Dataset",
        ],
        dtype={
            "Participant": np.int32,
            "Session": np.int32,
            "Age": np.int32,
            "Sex": str,
            "Pathology": str,
            "sEEG electrodes analysed": str,
            "Electrodes in seizure onset zone (SOZ)": str,
            "Handedness": str,
            "Date": str,
            "Session Start Time": str,
            "Dataset": str,
        },
    )
    df["Date"] = df["Date"] + " " + df["Session Start Time"]
    df["Date"] = df["Date"].apply(lambda date: datetime.strptime(date, "%y%m%d %H:%M"))
    df.drop(columns=["Session Start Time"], inplace=True)
    return df

In [166]:
def get_metadata_row(ctx: NixContext) -> pd.Series:
    df = _read_session_metadata()
    right_participant = df["Participant"] == ctx.subject
    right_session = df["Session"] == ctx.session
    right_dataset = df["Dataset"] == ctx.project
    return df.loc[right_participant & right_session & right_dataset].iloc[0]

In [167]:
metadata = get_metadata_row(nix_context)
electrodes_in_soz = metadata["Electrodes in seizure onset zone (SOZ)"].split(",")
print(electrodes_in_soz)

['AHR', 'LR']


In [193]:
ieeg_electrode_information = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays
)
print(ieeg_electrode_information)
print(ieeg_electrode_information["iEEG_Electrode_Map"][:])
print(
    ieeg_electrode_information["iEEG_Electrode_Map"].sources["iEEG electrode 5"].sources
)
print(ieeg_electrode_information["iEEG_Electrode_MNI_Coordinates"].shape)
print(ieeg_electrode_information["iEEG_Electrode_Manual_Entry"].dimensions[0].labels)
print(
    list(
        zip(
            ieeg_electrode_information["iEEG_Electrode_Map"][:],
            ieeg_electrode_information["iEEG_Electrode_Manual_Entry"]
            .dimensions[0]
            .labels,
        )
    )
)

[DataArray: {name = iEEG_Electrode_Map, type = nix.electrodes}, DataArray: {name = iEEG_Electrode_MNI_Coordinates, type = nix.electrodes.coordinates}, DataArray: {name = iEEG_Electrode_Manual_Entry, type = nix.electrodes.anatomical_labels}]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]
[Source: {name = mAHL5, type = nix.source}, Source: {name = STG, Left Superior Temporal Gyrus A22r, rostral area 22, type = nix.source}]
(48, 3)
('mAHL1', 'mAHL2', 'mAHL3', 'mAHL4', 'mAHL5', 'mAHL6', 'mAHL7', 'mAHL8', 'mAL1', 'mAL2', 'mAL3', 'mAL4', 'mAL5', 'mAL6', 'mAL7', 'mAL8', 'mECL1', 'mECL2', 'mECL3', 'mECL4', 'mECL5', 'mECL6', 'mECL7', 'mECL8', 'mLR1', 'mLR2', 'mLR3', 'mLR4', 'mLR5', 'mLR6', 'mLR7', 'mLR8', 'mPHL1', 'mPHL2', 'mPHL3', 'mPHL4', 'mPHL5', 'mPHL6', 'mPHL7', 'mPHL8', 'mPHR1', 'mPHR2', 'mPHR3', 'mPHR4', 'mPHR5', 'mPHR6', 'mPHR7', 'mPHR8')
[(1, 'mAHL1'), (2, 'mAHL2'), (3, 'mAHL3'), (4, 'mA

In [169]:
print(nixfile.blocks["Data_Subject_01_Session_01"].groups["Spike times"].data_arrays)

[DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_01, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_02, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_03, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_04, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_05, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_06, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_07, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_08, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_09, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_10, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_11, type = nix.irregular_sampled}, DataArray: {name = Spike_Times_Unit_1_uAHL_2_Trial_12

In [170]:
print(nixfile.sections["Session"].sections["Trial properties"].sections)

[Section: {name = Trial_01, type = nix.metadata.section}, Section: {name = Trial_02, type = nix.metadata.section}, Section: {name = Trial_03, type = nix.metadata.section}, Section: {name = Trial_04, type = nix.metadata.section}, Section: {name = Trial_05, type = nix.metadata.section}, Section: {name = Trial_06, type = nix.metadata.section}, Section: {name = Trial_07, type = nix.metadata.section}, Section: {name = Trial_08, type = nix.metadata.section}, Section: {name = Trial_09, type = nix.metadata.section}, Section: {name = Trial_10, type = nix.metadata.section}, Section: {name = Trial_11, type = nix.metadata.section}, Section: {name = Trial_12, type = nix.metadata.section}, Section: {name = Trial_13, type = nix.metadata.section}, Section: {name = Trial_14, type = nix.metadata.section}, Section: {name = Trial_15, type = nix.metadata.section}, Section: {name = Trial_16, type = nix.metadata.section}, Section: {name = Trial_17, type = nix.metadata.section}, Section: {name = Trial_18, typ

In [171]:
print(nixfile.sections["Subject"].props["Age"].name)
print(nixfile.sections["Subject"].props["Age"].values[0])

Age
24.0


In [172]:
print(
    len(
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["iEEG electrode information"]
        .data_arrays["iEEG_Electrode_Manual_Entry"]
        .dimensions[0]
        .labels
    )
)

48


In [173]:
print(
    len(
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Scalp EEG data"]
        .data_arrays["Scalp_EEG_Data_Trial_01"]
        .dimensions[0]
        .labels
    )
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"]
    .dimensions[0]
    .labels
)

19
('F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz', 'A1', 'A2')


In [174]:
print(
    nixfile.sections["Session"].sections["Trial properties"].sections["Trial_01"].props
)
print(
    nixfile.sections["Session"]
    .sections["Trial properties"]
    .sections["Trial_01"]
    .props["Response time"]
    .values[0]
)

[Property: {name = Trial number}, Property: {name = Set size}, Property: {name = Match}, Property: {name = Correct}, Property: {name = Response}, Property: {name = Response time}, Property: {name = Probe letter}, Property: {name = Artifact}]
2.484


In [175]:
tag = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags scalp EEG"]
    .tags[3]
)
print(tag.name, tag.position, tag.units, tag.extent)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays["Scalp_EEG_Data_Trial_01"][0]
)

Event_Probe_m2_H_Trial_01_Scalp_EEG (0.0, 0.0) ('s',) (9.4e-323, 0.0)
[-60.36156114  48.27822059  78.97903422 ...  72.58901499 108.25263725
 -86.62920055]


In [176]:
[
    (tag.name, tag.position, tag.extent)
    for tag in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags scalp EEG"]
    .tags
]

[('Event_Fixation_ss8_c1_m2_Trial_01_Scalp_EEG',
  (0.0, -6.0),
  (9.4e-323, 5e-324)),
 ('Event_Stimulus_ss8_xxxxxxxx_Trial_01_Scalp_EEG',
  (0.0, -5.0),
  (9.4e-323, 1e-323)),
 ('Event_Maintenance_Trial_01_Scalp_EEG', (0.0, -3.0), (9.4e-323, 1.5e-323)),
 ('Event_Probe_m2_H_Trial_01_Scalp_EEG', (0.0, 0.0), (9.4e-323, 0.0)),
 ('Event_Response_c1_Trial_01_Scalp_EEG', (0.0, 2.484), (9.4e-323, 0.0)),
 ('Event_Fixation_ss4_c1_m1_Trial_02_Scalp_EEG',
  (0.0, -6.0),
  (9.4e-323, 5e-324)),
 ('Event_Stimulus_ss4_xxxxxxxx_Trial_02_Scalp_EEG',
  (0.0, -5.0),
  (9.4e-323, 1e-323)),
 ('Event_Maintenance_Trial_02_Scalp_EEG', (0.0, -3.0), (9.4e-323, 1.5e-323)),
 ('Event_Probe_m1_T_Trial_02_Scalp_EEG', (0.0, 0.0), (9.4e-323, 0.0)),
 ('Event_Response_c1_Trial_02_Scalp_EEG', (0.0, 1.66775), (9.4e-323, 0.0)),
 ('Event_Fixation_ss8_c1_m2_Trial_03_Scalp_EEG',
  (0.0, -6.0),
  (9.4e-323, 5e-324)),
 ('Event_Stimulus_ss8_xxxxxxxx_Trial_03_Scalp_EEG',
  (0.0, -5.0),
  (9.4e-323, 1e-323)),
 ('Event_Maintenance_

In [177]:
[
    (tag.name, tag.position, tag.extent)
    for tag in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags iEEG"]
    .tags
]

[('Event_Fixation_ss8_c1_m2_Trial_01_iEEG', (0.0, -6.0), (2.37e-322, 5e-324)),
 ('Event_Stimulus_ss8_xxxxxxxx_Trial_01_iEEG',
  (0.0, -5.0),
  (2.37e-322, 1e-323)),
 ('Event_Maintenance_Trial_01_iEEG', (0.0, -3.0), (2.37e-322, 1.5e-323)),
 ('Event_Probe_m2_H_Trial_01_iEEG', (0.0, 0.0), (2.37e-322, 0.0)),
 ('Event_Response_c1_Trial_01_iEEG', (0.0, 2.484), (2.37e-322, 0.0)),
 ('Event_Fixation_ss4_c1_m1_Trial_02_iEEG', (0.0, -6.0), (2.37e-322, 5e-324)),
 ('Event_Stimulus_ss4_xxxxxxxx_Trial_02_iEEG',
  (0.0, -5.0),
  (2.37e-322, 1e-323)),
 ('Event_Maintenance_Trial_02_iEEG', (0.0, -3.0), (2.37e-322, 1.5e-323)),
 ('Event_Probe_m1_T_Trial_02_iEEG', (0.0, 0.0), (2.37e-322, 0.0)),
 ('Event_Response_c1_Trial_02_iEEG', (0.0, 1.66775), (2.37e-322, 0.0)),
 ('Event_Fixation_ss8_c1_m2_Trial_03_iEEG', (0.0, -6.0), (2.37e-322, 5e-324)),
 ('Event_Stimulus_ss8_xxxxxxxx_Trial_03_iEEG',
  (0.0, -5.0),
  (2.37e-322, 1e-323)),
 ('Event_Maintenance_Trial_03_iEEG', (0.0, -3.0), (2.37e-322, 1.5e-323)),
 ('Even

In [178]:
sum(
    [
        (a.position[1] - b.position[1])
        for (a, b) in zip(
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Trial events single tags iEEG"]
        .tags,
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Trial events single tags scalp EEG"]
        .tags,
    )
    ]
)

0.0

In [179]:
print(
    tag[:]
    for tag in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags scalp EEG"]
    .tags
)

<generator object <genexpr> at 0x000002CBDFCD9F50>


In [180]:
[
    (
        trial.props["Probe letter"].values[0],
        trial.props["Match"].values[0],
        trial.props["Response"].values[0],
        bool(trial.props["Correct"].values[0]),
    )
    for trial in nixfile.sections["Session"].sections["Trial properties"].sections
]

[('H', 2.0, 52.0, True),
 ('T', 1.0, 51.0, True),
 ('L', 2.0, 52.0, True),
 ('G', 2.0, 52.0, True),
 ('S', 2.0, 52.0, True),
 ('R', 1.0, 51.0, True),
 ('N', 2.0, 52.0, True),
 ('Z', 1.0, 51.0, True),
 ('Z', 2.0, 52.0, True),
 ('V', 2.0, 52.0, True),
 ('G', 2.0, 52.0, True),
 ('D', 1.0, 51.0, True),
 ('L', 2.0, 52.0, True),
 ('B', 1.0, 2.0, False),
 ('Q', 2.0, 52.0, True),
 ('C', 1.0, 51.0, True),
 ('B', 2.0, 52.0, True),
 ('D', 2.0, 52.0, True),
 ('D', 2.0, 52.0, True),
 ('D', 2.0, 52.0, True),
 ('Q', 2.0, 52.0, True),
 ('D', 2.0, 52.0, True),
 ('B', 1.0, 2.0, False),
 ('C', 1.0, 51.0, True),
 ('P', 1.0, 51.0, True),
 ('P', 1.0, 51.0, True),
 ('W', 2.0, 52.0, True),
 ('B', 1.0, 51.0, True),
 ('W', 1.0, 51.0, True),
 ('D', 2.0, 52.0, True),
 ('Q', 2.0, 52.0, True),
 ('P', 1.0, 51.0, True),
 ('F', 2.0, 52.0, True),
 ('V', 1.0, 51.0, True),
 ('D', 1.0, 2.0, False),
 ('Q', 1.0, 51.0, True),
 ('T', 2.0, 52.0, True),
 ('T', 1.0, 51.0, True),
 ('C', 2.0, 1.0, False),
 ('V', 2.0, 52.0, True),


In [181]:
[
    trial.dimensions[1].sampling_interval * trial.shape[1] + trial.dimensions[1].offset
    for trial in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Scalp EEG data"]
    .data_arrays
]

[2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0]

In [4]:
[
    (tag.name, tag.position, tag.extent)
    for tag in nixfile.blocks["Data_Subject_01_Session_04"]
    .groups["Trial events single tags spike times"]
    .tags
]

[]

In [183]:
print(nixfile.blocks["Data_Subject_01_Session_01"].groups)

[Group: {name = iEEG data, type = nix.group}, Group: {name = Scalp EEG data, type = nix.group}, Group: {name = Trial events single tags iEEG, type = nix.group}, Group: {name = Trial events single tags scalp EEG, type = nix.group}, Group: {name = Spike times, type = nix.group}, Group: {name = Spike waveforms, type = nix.group}, Group: {name = Trial events single tags spike times, type = nix.group}, Group: {name = Spike times multitags, type = nix.group}, Group: {name = iEEG electrode information, type = nix.group}, Group: {name = Scalp EEG electrode information, type = nix.group}]


In [184]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike waveforms"]
    .data_arrays[0]
    .shape
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike waveforms"]
    .data_arrays[0][:]
)
print(
    len(
        nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Spike waveforms"]
        .data_arrays
    )
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"].groups["Spike waveforms"].data_arrays
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike waveforms"]
    .data_arrays[0]
    .dimensions[1]
    .sampling_interval
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays["iEEG_Electrode_Manual_Entry"]
    .dimensions[0]
    .labels
)

(2, 64)
[[-1.15498519 -1.2420795  -1.39041018 -1.59339774 -1.80332696 -1.95180058
  -2.01095414 -2.05641198 -2.26199579 -2.77834821 -3.53060865 -4.065063
  -3.59830332 -1.32600892  3.12768221  9.37457848 16.27382278 22.33140373
  26.28946304 27.59065056 26.49193192 23.81641197 20.52307129 17.33119774
  14.55776119 12.18099022 10.02907562  7.9614172   5.95565939  4.0891242
   2.46191764  1.1252358   0.05545247 -0.82347256 -1.59405041 -2.30593109
  -2.96202755 -3.53175974 -3.97912788 -4.28814363 -4.47329521 -4.5731535
  -4.63381433 -4.69262791 -4.77015352 -4.8713007  -4.98986244 -5.11095142
  -5.21228981 -5.26962852 -5.26778078 -5.21076727 -5.12189436 -5.03122997
  -4.9581275  -4.90169859 -4.84601212 -4.77483273 -4.68258286 -4.5728364
  -4.447896   -4.30197144 -4.12572622 -3.91771102]
 [ 4.94364023  4.91290569  4.91977644  5.01283836  5.2883482   5.75018406
   6.26473713  6.68398905  6.89293432  6.78553247  6.37321568  5.96807718
   5.96840382  6.1736269   5.90326643  4.85850191  3.64075

In [185]:
print(
    [
        times.name
        for times in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike times"]
    .data_arrays
    ]
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"].groups["Spike times"].data_arrays[0][:]
)

['Spike_Times_Unit_1_uAHL_2_Trial_01', 'Spike_Times_Unit_1_uAHL_2_Trial_02', 'Spike_Times_Unit_1_uAHL_2_Trial_03', 'Spike_Times_Unit_1_uAHL_2_Trial_04', 'Spike_Times_Unit_1_uAHL_2_Trial_05', 'Spike_Times_Unit_1_uAHL_2_Trial_06', 'Spike_Times_Unit_1_uAHL_2_Trial_07', 'Spike_Times_Unit_1_uAHL_2_Trial_08', 'Spike_Times_Unit_1_uAHL_2_Trial_09', 'Spike_Times_Unit_1_uAHL_2_Trial_10', 'Spike_Times_Unit_1_uAHL_2_Trial_11', 'Spike_Times_Unit_1_uAHL_2_Trial_12', 'Spike_Times_Unit_1_uAHL_2_Trial_13', 'Spike_Times_Unit_1_uAHL_2_Trial_14', 'Spike_Times_Unit_1_uAHL_2_Trial_15', 'Spike_Times_Unit_1_uAHL_2_Trial_16', 'Spike_Times_Unit_1_uAHL_2_Trial_17', 'Spike_Times_Unit_1_uAHL_2_Trial_18', 'Spike_Times_Unit_1_uAHL_2_Trial_19', 'Spike_Times_Unit_1_uAHL_2_Trial_20', 'Spike_Times_Unit_1_uAHL_2_Trial_21', 'Spike_Times_Unit_1_uAHL_2_Trial_22', 'Spike_Times_Unit_1_uAHL_2_Trial_23', 'Spike_Times_Unit_1_uAHL_2_Trial_24', 'Spike_Times_Unit_1_uAHL_2_Trial_25', 'Spike_Times_Unit_1_uAHL_2_Trial_26', 'Spike_Time

In [186]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike times multitags"]
    .multi_tags[0]
    .features[0]
    .data[0]
)

[-1.15498519 -1.2420795  -1.39041018 -1.59339774 -1.80332696 -1.95180058
 -2.01095414 -2.05641198 -2.26199579 -2.77834821 -3.53060865 -4.065063
 -3.59830332 -1.32600892  3.12768221  9.37457848 16.27382278 22.33140373
 26.28946304 27.59065056 26.49193192 23.81641197 20.52307129 17.33119774
 14.55776119 12.18099022 10.02907562  7.9614172   5.95565939  4.0891242
  2.46191764  1.1252358   0.05545247 -0.82347256 -1.59405041 -2.30593109
 -2.96202755 -3.53175974 -3.97912788 -4.28814363 -4.47329521 -4.5731535
 -4.63381433 -4.69262791 -4.77015352 -4.8713007  -4.98986244 -5.11095142
 -5.21228981 -5.26962852 -5.26778078 -5.21076727 -5.12189436 -5.03122997
 -4.9581275  -4.90169859 -4.84601212 -4.77483273 -4.68258286 -4.5728364
 -4.447896   -4.30197144 -4.12572622 -3.91771102]


In [27]:
nixfile.sections["Session"].props["Trial duration"].values[0]

8.0

In [28]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays["iEEG_Electrode_Map"]
    .sources[0]
    .sources[0]
    .name
)

mAHL1


# Amygdala

In [89]:
nix_context = NixContext(
    session=1, subject=1, project="Human_Amygdala_MUA_sEEG_FearVideo"
)
nixfile = read_nix(nix_context)

In [30]:
print(
    [
        (trial.props["Condition"].values[0], trial.props["Condition name"].values[0])
        for trial in nixfile.sections["Session"].sections["Trial properties"].sections
    ]
)

[(1.0, 'Neutral'), (2.0, 'Aversive'), (1.0, 'Neutral'), (2.0, 'Aversive'), (1.0, 'Neutral'), (2.0, 'Aversive'), (1.0, 'Neutral'), (2.0, 'Aversive'), (1.0, 'Neutral'), (2.0, 'Aversive'), (1.0, 'Neutral'), (2.0, 'Aversive'), (1.0, 'Neutral'), (2.0, 'Aversive'), (1.0, 'Neutral'), (2.0, 'Aversive'), (1.0, 'Neutral')]


In [31]:
print(
    [
        channel.sources[2].name
        for channel in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays["iEEG_Electrode_Map"]
    .sources
    ]
)

['Outside SOZ', 'Inside SOZ']


In [32]:
spike_events = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags spike times"]
    .tags
)
ieeg_events = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags iEEG"]
    .tags
)
print(
    [
        (spike_event.name, ieeg_event.name)
        for spike_event, ieeg_event in zip(spike_events, ieeg_events)
    ]
)
print([(tag.name, tag.position, tag.extent) for tag in spike_events])

[('Stimulus_Condition_1_Neutral_Trial_01_Spike_Times', 'Stimulus_Condition_1_Neutral_Trial_01_iEEG'), ('Stimulus_Condition_2_Aversive_Trial_02_Spike_Times', 'Stimulus_Condition_2_Aversive_Trial_02_iEEG'), ('Stimulus_Condition_1_Neutral_Trial_03_Spike_Times', 'Stimulus_Condition_1_Neutral_Trial_03_iEEG'), ('Stimulus_Condition_2_Aversive_Trial_04_Spike_Times', 'Stimulus_Condition_2_Aversive_Trial_04_iEEG'), ('Stimulus_Condition_1_Neutral_Trial_05_Spike_Times', 'Stimulus_Condition_1_Neutral_Trial_05_iEEG'), ('Stimulus_Condition_2_Aversive_Trial_06_Spike_Times', 'Stimulus_Condition_2_Aversive_Trial_06_iEEG'), ('Stimulus_Condition_1_Neutral_Trial_07_Spike_Times', 'Stimulus_Condition_1_Neutral_Trial_07_iEEG'), ('Stimulus_Condition_2_Aversive_Trial_08_Spike_Times', 'Stimulus_Condition_2_Aversive_Trial_08_iEEG'), ('Stimulus_Condition_1_Neutral_Trial_09_Spike_Times', 'Stimulus_Condition_1_Neutral_Trial_09_iEEG'), ('Stimulus_Condition_2_Aversive_Trial_10_Spike_Times', 'Stimulus_Condition_2_Avers

In [33]:
print(
    [
        spike.name
        for spike in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike waveforms"]
    .data_arrays
    ]
)

['Spike_Waveform_Unit_1_uAL_1', 'Spike_Waveform_Unit_2_uAL_2', 'Spike_Waveform_Unit_3_uAL_6', 'Spike_Waveform_Unit_4_uAR_7']


In [34]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"].groups["iEEG data"].data_arrays[0][:]
)
print(
    nixfile.blocks["Data_Subject_01_Session_01"].groups["Spike times"].data_arrays[0][:]
)

[[-12.38574954 -19.38915739 -18.24174916 ... -12.61341031 -11.57805962
   -8.81919546]
 [ -2.08570364  -4.01717376  -5.15755012 ...  10.42427998   8.62187088
    8.98090842]]
[-1.5615985  -1.3829735  -1.372286   -1.31462975 -1.01050475 -0.99031725
 -0.792036   -0.77994225 -0.774411   -0.21450475  2.06336925  2.4005255
  2.845713    2.8531505   5.039837    5.2580245   5.5073985   5.7853985
  5.97024225  5.986336    6.7391485   6.7933985   7.872085    7.87836625
  7.88586625  8.93786625  8.94549125 10.27696    10.28211625 10.30355375
 10.4748965  10.480084   10.5050215  10.507459   10.5196465  10.52749025
 11.015709   12.12392775 14.0591455  15.5138945  19.660081   20.0820185
 20.649955   23.068454   23.108954   23.114829   23.13142275]


In [35]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags iEEG"]
    .tags[4]
    .position
)

(0.0, 0.0)


In [36]:
print(
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays["iEEG_Electrode_Map"]
    .sources[0]
    .sources[0]
    .name,
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays["iEEG_Electrode_Map"]
    .sources[1]
    .sources[0]
    .name,
)

mAL2-mAL1 mAR2-mAR1


In [96]:
_general = nixfile.sections["General"]
recording_location = _general.props["Recording location"].values[0]
recording_setup = (
    _general.sections["Recording setup"].props["Recording setup iEEG"].values[0]
)

_electrode_map = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG electrode information"]
    .data_arrays["iEEG_Electrode_MNI_Coordinates"]
)

# See <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
# in x, y ,z
eeg_electrode_locations = np.ndarray(_electrode_map.shape)
_electrode_map.read_direct(eeg_electrode_locations)
# BESA map: +X is nose, +Y is left, +Z is up according to <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
eeg_electrode_locations_df = pd.DataFrame(
    eeg_electrode_locations, columns=["X", "Y", "Z"]
)
# By eye: ["F3", "F4", "C3", "C4", "P3", "P4", "O1", "O2", "F7", "F8", "T7", "T8", "P7", "P8", "FZ", "CZ", "PZ", "LPA", "RPA"]
_labels = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["iEEG data"]
    .data_arrays["iEEG_Data_Trial_01"]
    .dimensions[0]
    .labels
)
eeg_electrode_locations_df.insert(3, "location", _labels)
print(eeg_electrode_locations_df.to_string())

       X     Y      Z   location
0 -25.16 -0.06 -19.27  mAL2-mAL1
1  22.01 -3.91 -24.98  mAR2-mAR1


# Visual WM

In [18]:
nix_context = NixContext(
    session=1,
    subject=2,
    project="Human_MTL_units_visual_WM",
)
nixfile = read_nix(nix_context)

In [196]:
print([block.name for block in nixfile.blocks])
print([section.name for section in nixfile.sections])

['Data_Subject_01_Session_01']
['General', 'Task', 'Subject', 'Session']


In [197]:
print([section.name for section in nixfile.sections["Session"].props])
print([section.name for section in nixfile.sections["Session"].sections])
print([prop.name for prop in nixfile.sections["Subject"].props])
print([section.name for section in nixfile.sections["Subject"].sections])
print([prop.name for prop in nixfile.sections["Task"].props])
print([section.name for section in nixfile.sections["Task"].sections])
print([prop.name for prop in nixfile.sections["General"].props])
print([section.name for section in nixfile.sections["General"].sections])

['Number of trials', 'Trial duration']
['Trial properties']
['Age', 'Sex', 'Pathology', 'Handedness', 'Implanted electrodes', 'Electrodes in seizure onset zone (SOZ)']
[]
['Task name', 'Task description']
[]
['Institution', 'Recording location']
['Related publications', 'Recording setup']


In [22]:
print(
    [
        (prop.name, prop.values[0])
        for prop in nixfile.sections["Session"]
    .sections["Trial properties"]
    .sections[-1]
    .props
    ]
)

[('Trial number', 192.0), ('Set size', 4.0), ('Match', 1.0), ('Correct', 1.0), ('Response', 1.0), ('Response time', 0.6675)]


In [199]:
print(
    [
        prop.name
        for prop in nixfile.sections["General"].sections["Recording setup"].props
    ]
)
print(
    nixfile.sections["General"]
    .sections["Recording setup"]
    .props["Recording setup iEEG"]
    .values[0]
)

['Recording setup iEEG']
Neuralynx ATLAS; original sampling rate, 32 kHz; passband, 1 to 8000 Hz


In [231]:
print([group.name for group in nixfile.blocks["Data_Subject_01_Session_01"].groups])

['Spike times', 'Spike waveforms', 'Trial events single tags spike times', 'Spike times multitags']


In [201]:
print(
    "\n".join(
        [
            f"{tag.name}: {tag.position}, {tag.extent}, {tag.sources}"
            for tag in nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Trial events single tags spike times"]
        .tags
        ]
    )
)

Event_Fixation_ss2_c1_m1_Trial_01_Spike_Times: (-1.7,), (0.9,), []
Event_Memory_Array_ss2_Trial_01_Spike_Times: (-0.8,), (0.8,), []
Event_Retention_Trial_01_Spike_Times: (0.0,), (0.9,), []
Event_Probe_m1_Trial_01_Spike_Times: (0.9,), (0.0,), []
Event_Response_c1_Trial_01_Spike_Times: (1.81875,), (0.0,), []
Event_Fixation_ss4_c0_m2_Trial_02_Spike_Times: (-1.7,), (0.9,), []
Event_Memory_Array_ss4_Trial_02_Spike_Times: (-0.8,), (0.8,), []
Event_Retention_Trial_02_Spike_Times: (0.0,), (0.9,), []
Event_Probe_m2_Trial_02_Spike_Times: (0.9,), (0.0,), []
Event_Response_c0_Trial_02_Spike_Times: (1.7725,), (0.0,), []
Event_Fixation_ss6_c1_m2_Trial_03_Spike_Times: (-1.7,), (0.9,), []
Event_Memory_Array_ss6_Trial_03_Spike_Times: (-0.8,), (0.8,), []
Event_Retention_Trial_03_Spike_Times: (0.0,), (0.9,), []
Event_Probe_m2_Trial_03_Spike_Times: (0.9,), (0.0,), []
Event_Response_c1_Trial_03_Spike_Times: (1.721,), (0.0,), []
Event_Fixation_ss1_c1_m2_Trial_04_Spike_Times: (-1.7,), (0.9,), []
Event_Memory

In [202]:
print(
    "\n".join(
        [
            f"{tag.name}: {[pos[0] for pos in tag.positions]}"
            for tag in nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Spike times multitags"]
        .multi_tags
        ]
    )
)

Multitag_Spike_Times_Unit_1_uAHL_2_Trial_01: []
Multitag_Spike_Times_Unit_2_uAHL_2_Trial_01: [-1.44482725, -1.389046, -1.36132725, -1.0062335, -0.7864835, -0.72276475, -0.6261085, -0.36989074999999993, -0.233047, -0.20373449999999993, 0.08448425000000004, 0.5585467499999999, 0.8674842500000002, 1.1287654999999999, 1.3573905, 1.9223592500000002, 2.0062957500000005, 2.5684519999999997]
Multitag_Spike_Times_Unit_3_uAHL_2_Trial_01: [-1.2339835, -0.05020325000000003, 0.12929674999999996, 0.20020300000000013, 0.4004842500000001, 0.4050467500000001, 1.590578, 1.5942655]
Multitag_Spike_Times_Unit_4_uAHL_2_Trial_01: []
Multitag_Spike_Times_Unit_5_uAHL_2_Trial_01: [-1.205171, 0.02529675000000009]
Multitag_Spike_Times_Unit_6_uAHL_2_Trial_01: [0.5271405, 1.23548425]
Multitag_Spike_Times_Unit_7_uAHL_2_Trial_01: [-0.027453249999999985, 0.09660924999999998, 0.09860924999999998, 0.1762030000000001, 0.29526549999999996, 0.3168279999999999, 0.39764049999999984, 0.40910925, 0.4149217500000002, 1.57657800

In [203]:
print(
    "\n".join(
        [
            f"{data.name}: {data[:]}"
            for data in nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Spike times"]
        .data_arrays
        ]
    )
)

Spike_Times_Unit_1_uAHL_2_Trial_01: []
Spike_Times_Unit_1_uAHL_2_Trial_02: [0.85210925 0.85673425]
Spike_Times_Unit_1_uAHL_2_Trial_03: []
Spike_Times_Unit_1_uAHL_2_Trial_04: [-0.8740455  -0.87126425 -0.86576425 -0.72613925  0.616391    0.61923475
  0.62279725]
Spike_Times_Unit_1_uAHL_2_Trial_05: [0.01951625 0.0260475 ]
Spike_Times_Unit_1_uAHL_2_Trial_06: [0.93873375]
Spike_Times_Unit_1_uAHL_2_Trial_07: []
Spike_Times_Unit_1_uAHL_2_Trial_08: [0.653797   0.660047   0.6627345  1.642421   1.64495225 1.64751475
 1.652046   1.65507725 1.658671  ]
Spike_Times_Unit_1_uAHL_2_Trial_09: [2.426983   2.432608   2.43463925 2.43688925 2.438483   2.44163925
 2.446983  ]
Spike_Times_Unit_1_uAHL_2_Trial_10: [-0.368203    1.8129835   1.81682725]
Spike_Times_Unit_1_uAHL_2_Trial_11: []
Spike_Times_Unit_1_uAHL_2_Trial_12: [-1.4622335  -0.565796    0.69467175  0.696453    0.7136405   0.71754675
  0.7198905   0.72310925  0.726078    0.73117175  2.5782645   2.5840145
  2.58610825]
Spike_Times_Unit_1_uAHL_2_Tri

In [232]:
print(
    "\n".join(
        [
            f"{data.name}: {data[:]}"
            for data in nixfile.blocks["Data_Subject_01_Session_01"]
        .groups["Spike waveforms"]
        .data_arrays
        ]
    )
)

Spike_Waveform_Unit_1_uAHL_2: [[ 0.74753606  1.14389265  1.51171374  1.89440286  2.35166502  2.94508338
   3.70534396  4.59468126  5.49910927  6.28543806  6.92967749  7.67033482
   9.08405972 11.96734715 16.98355865 24.19270897 32.73246384 40.92310715
  46.82782745 48.96546173 46.80134583 40.87591171 32.58952332 23.68756676
  15.63602543  9.21644783  4.52057362  1.22919118 -1.07115901 -2.70294881
  -3.86376095 -4.66783857 -5.1897378  -5.48915339 -5.62508631 -5.66076803
  -5.65658283 -5.6582675  -5.69350386 -5.77897739 -5.92701817 -6.14141512
  -6.40658331 -6.68589306 -6.9401269  -7.15634298 -7.36134768 -7.6011672
  -7.89702129 -8.21475601 -8.47846794 -8.62245369 -8.64282322 -8.60899639
  -8.62324715 -8.75455666 -8.99292564 -9.25456047 -9.43083286 -9.44636345
  -9.28970051 -9.00172138 -8.63681126 -8.2307663 ]
 [15.67541981 15.74547863 15.84705067 16.0099659  16.22278595 16.42961502
  16.58048439 16.71883965 16.95892525 17.34599876 17.79533958 18.13002586
  18.08055496 17.30920792 15.711

In [236]:
wave_names = [
    data.name
    for data in nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Spike waveforms"]
    .data_arrays
]
wave_names.sort()
print(wave_names)
print(len(wave_names))

['Spike_Waveform_Unit_10_uAHL_5', 'Spike_Waveform_Unit_11_uAHL_5', 'Spike_Waveform_Unit_12_uAHL_5', 'Spike_Waveform_Unit_13_uAHL_7', 'Spike_Waveform_Unit_14_uAHL_7', 'Spike_Waveform_Unit_15_uAHL_7', 'Spike_Waveform_Unit_16_uAHL_8', 'Spike_Waveform_Unit_17_uAHL_8', 'Spike_Waveform_Unit_18_uAHL_8', 'Spike_Waveform_Unit_19_uAHL_8', 'Spike_Waveform_Unit_1_uAHL_2', 'Spike_Waveform_Unit_20_uAHR_2', 'Spike_Waveform_Unit_21_uAHR_6', 'Spike_Waveform_Unit_22_uAL_1', 'Spike_Waveform_Unit_23_uAR_7', 'Spike_Waveform_Unit_24_uAR_7', 'Spike_Waveform_Unit_25_uAR_7', 'Spike_Waveform_Unit_26_uECL_1', 'Spike_Waveform_Unit_27_uECL_1', 'Spike_Waveform_Unit_28_uECL_1', 'Spike_Waveform_Unit_29_uECL_1', 'Spike_Waveform_Unit_2_uAHL_2', 'Spike_Waveform_Unit_30_uECL_3', 'Spike_Waveform_Unit_31_uPHR_5', 'Spike_Waveform_Unit_32_uPHR_5', 'Spike_Waveform_Unit_33_uPHR_5', 'Spike_Waveform_Unit_3_uAHL_2', 'Spike_Waveform_Unit_4_uAHL_2', 'Spike_Waveform_Unit_5_uAHL_2', 'Spike_Waveform_Unit_6_uAHL_2', 'Spike_Waveform_Uni

In [207]:
session = nixfile.blocks["Data_Subject_01_Session_01"]
events = session.groups["Trial events single tags spike times"].tags
print(events)

[Tag: {name = Event_Fixation_ss2_c1_m1_Trial_01_Spike_Times, type = nix.events}, Tag: {name = Event_Memory_Array_ss2_Trial_01_Spike_Times, type = nix.events}, Tag: {name = Event_Retention_Trial_01_Spike_Times, type = nix.events}, Tag: {name = Event_Probe_m1_Trial_01_Spike_Times, type = nix.events}, Tag: {name = Event_Response_c1_Trial_01_Spike_Times, type = nix.events}, Tag: {name = Event_Fixation_ss4_c0_m2_Trial_02_Spike_Times, type = nix.events}, Tag: {name = Event_Memory_Array_ss4_Trial_02_Spike_Times, type = nix.events}, Tag: {name = Event_Retention_Trial_02_Spike_Times, type = nix.events}, Tag: {name = Event_Probe_m2_Trial_02_Spike_Times, type = nix.events}, Tag: {name = Event_Response_c0_Trial_02_Spike_Times, type = nix.events}, Tag: {name = Event_Fixation_ss6_c1_m2_Trial_03_Spike_Times, type = nix.events}, Tag: {name = Event_Memory_Array_ss6_Trial_03_Spike_Times, type = nix.events}, Tag: {name = Event_Retention_Trial_03_Spike_Times, type = nix.events}, Tag: {name = Event_Probe_m

In [243]:
_events = (
    nixfile.blocks["Data_Subject_01_Session_01"]
    .groups["Trial events single tags scalp EEG"]
    .tags
)
_EVENTS_PER_TRIAL = 5
trials = 1
for i in range(0, trials * _EVENTS_PER_TRIAL, _EVENTS_PER_TRIAL):
    print("--")
    for j in range(_EVENTS_PER_TRIAL):
        _event = _events[i + j]
        print(
            f"Event {_event.name}: {_event.position[1]} {_event.units[0]} (w.r.t. fixation), lasted {_event.extent[1]} {_event.units[0]}"
        )

KeyError: "Item not found 'Data_Subject_01_Session_01'"

In [250]:
print(max([max(data[:]) for data in session.groups["Spike times"].data_arrays if len(data[:]) > 0]))

2.8999532500000003


In [242]:
max_responses = []
for i in range(13):
    nix_context = NixContext(
        session=1,
        subject=i + 1,
        project="Human_MTL_units_visual_WM",
    )
nixfile = read_nix(nix_context)
max_responses.append(max(
    [
        section.props["Response time"]
        .values[0]
        for section in
        nixfile.sections["Session"]
        .sections["Trial properties"]
        .sections
    ])
)
print(max_responses)

[2.1442499999999995, 2.30025, 2.51, 2.407375, 2.88125, 2.5277499999999997, 1.90525, 2.9665, 3.14575, 2.85375, 2.05175, 2.85, 1.7807500000000003]
